In [2]:
spark = SparkSession.builder \
.appName("RDD_Operations")\
.getOrCreate()

25/08/14 05:42:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
customer_data = [
    "customer_id,name,city,state,country,registration_date,is_active",
    "0,Customer_0,Pune,Maharashtra,India,2023-06-29,False",
    "1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True",
    "2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True",
    "3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False",
    "4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False",
    "5,Customer_5,Hyderabad,Karnataka,India,2023-07-28,False"
]

In [8]:
data_rdd = spark.sparkContext.parallelize(customer_data)

In [9]:
header = data_rdd.first()

data_rdd =data_rdd.filter(lambda row : row!=header)

In [10]:
data_rdd.collect()

['0,Customer_0,Pune,Maharashtra,India,2023-06-29,False',
 '1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True',
 '2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True',
 '3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False',
 '4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False',
 '5,Customer_5,Hyderabad,Karnataka,India,2023-07-28,False']

In [11]:
cities_rdd = data_rdd.map(lambda row:(row.split(',')[2],1))


In [12]:
cities_rdd.collect()

[('Pune', 1),
 ('Bangalore', 1),
 ('Hyderabad', 1),
 ('Bangalore', 1),
 ('Ahmedabad', 1),
 ('Hyderabad', 1)]

In [13]:
customers_per_city = cities_rdd.reduceByKey(lambda x,y:x+y)

In [14]:
customers_per_city.collect()

[('Pune', 1), ('Hyderabad', 2), ('Bangalore', 2), ('Ahmedabad', 1)]

In [17]:
customers_per_city.take(4)

[('Pune', 1), ('Hyderabad', 2), ('Bangalore', 2), ('Ahmedabad', 1)]

In [20]:
 data_rdd.filter(lambda row :row.split(',')[6]=='True').collect()

['1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True',
 '2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True']

In [22]:
data_rdd.map(lambda row :(row.split(',')[2],1)).reduceByKey(lambda x,y : x+y).count()

4

# GroupByKey and ReduceByKey 

In [24]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("ReduceByKey_vs_GroupByKey") \
    .getOrCreate()

25/08/14 12:19:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [28]:
!hadoop fs -ls /user/donnirmal86

Found 6 items
drwxr-xr-x   - donnirmal86 hadoop          0 2025-08-08 11:06 /user/donnirmal86/.sparkStaging
-rw-r--r--   2 donnirmal86 hadoop       5488 2025-08-13 09:41 /user/donnirmal86/first_100_customers.csv
-rw-r--r--   2 donnirmal86 hadoop     836123 2025-08-13 09:43 /user/donnirmal86/items.csv
-rw-r--r--   2 donnirmal86 hadoop     863301 2025-08-13 09:43 /user/donnirmal86/orders.csv
-rw-r--r--   2 donnirmal86 hadoop     854480 2025-08-13 09:44 /user/donnirmal86/payments.csv
-rw-r--r--   2 donnirmal86 hadoop     769971 2025-08-13 09:44 /user/donnirmal86/shippings.csv


In [30]:
!hadoop fs -head /user/donnirmal86/first_100_customers.csv

customer_id,name,city,state,country,registration_date,is_active
0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False
5,Customer_5,Hyderabad,Karnataka,India,2023-07-28,False
6,Customer_6,Pune,Delhi,India,2023-08-29,False
7,Customer_7,Ahmedabad,West Bengal,India,2023-12-28,True
8,Customer_8,Pune,Karnataka,India,2023-06-22,True
9,Customer_9,Mumbai,Telangana,India,2023-01-05,True
10,Customer_10,Pune,Gujarat,India,2023-08-05,True
11,Customer_11,Delhi,West Bengal,India,2023-08-02,False
12,Customer_12,Chennai,Gujarat,India,2023-11-21,False
13,Customer_13,Chennai,Karnataka,India,2023-11-06,True
14,Customer_14,Hyderabad,Tamil Nadu,India,2023-02-07,False
15,Customer_15,Mumbai,Gujarat,India,2023-03-02,True
16,Customer_16,Chennai,Karnataka,India,2023-04-05,False
17,Customer_17,Hyd

In [31]:
hdfs_path ='/user/donnirmal86/first_100_customers.csv'
rdd = spark.sparkContext.textFile(hdfs_path)

In [35]:
header = rdd.first()
rdd_no_header = rdd.filter(lambda row : row!=header).map(lambda row :row.split(','))

In [36]:
rdd_no_header.collect()

[['0', 'Customer_0', 'Pune', 'Maharashtra', 'India', '2023-06-29', 'False'],
 ['1', 'Customer_1', 'Bangalore', 'Tamil Nadu', 'India', '2023-12-07', 'True'],
 ['2', 'Customer_2', 'Hyderabad', 'Gujarat', 'India', '2023-10-27', 'True'],
 ['3', 'Customer_3', 'Bangalore', 'Karnataka', 'India', '2023-10-17', 'False'],
 ['4', 'Customer_4', 'Ahmedabad', 'Karnataka', 'India', '2023-03-14', 'False'],
 ['5', 'Customer_5', 'Hyderabad', 'Karnataka', 'India', '2023-07-28', 'False'],
 ['6', 'Customer_6', 'Pune', 'Delhi', 'India', '2023-08-29', 'False'],
 ['7',
  'Customer_7',
  'Ahmedabad',
  'West Bengal',
  'India',
  '2023-12-28',
  'True'],
 ['8', 'Customer_8', 'Pune', 'Karnataka', 'India', '2023-06-22', 'True'],
 ['9', 'Customer_9', 'Mumbai', 'Telangana', 'India', '2023-01-05', 'True'],
 ['10', 'Customer_10', 'Pune', 'Gujarat', 'India', '2023-08-05', 'True'],
 ['11', 'Customer_11', 'Delhi', 'West Bengal', 'India', '2023-08-02', 'False'],
 ['12', 'Customer_12', 'Chennai', 'Gujarat', 'India', '202

In [37]:
rdd_no_header.first()

['0', 'Customer_0', 'Pune', 'Maharashtra', 'India', '2023-06-29', 'False']

In [43]:
reduced_rdd = rdd_no_header.map(lambda row:(row[2],1)).reduceByKey(lambda x,y :x+y)

In [44]:
reduced_rdd.collect()

[('Pune', 23),
 ('Hyderabad', 12),
 ('Mumbai', 9),
 ('Delhi', 9),
 ('Bangalore', 7),
 ('Ahmedabad', 12),
 ('Chennai', 12),
 ('Kolkata', 15)]

In [48]:
##groupByKey `

grouped_rdd = rdd_no_header.map(lambda row :(row[2],1)).groupByKey()

In [49]:
grouped_rdd.collect()  ## Here the result shows resultiterbale like pune,1,1,1,so instead of doing this its shows result iterable.

[('Pune', <pyspark.resultiterable.ResultIterable at 0x7f1efd26b190>),
 ('Hyderabad', <pyspark.resultiterable.ResultIterable at 0x7f1efd3da350>),
 ('Mumbai', <pyspark.resultiterable.ResultIterable at 0x7f1efd26a750>),
 ('Delhi', <pyspark.resultiterable.ResultIterable at 0x7f1efd26a8d0>),
 ('Bangalore', <pyspark.resultiterable.ResultIterable at 0x7f1efd26af90>),
 ('Ahmedabad', <pyspark.resultiterable.ResultIterable at 0x7f1efd4f34d0>),
 ('Chennai', <pyspark.resultiterable.ResultIterable at 0x7f1efd26a950>),
 ('Kolkata', <pyspark.resultiterable.ResultIterable at 0x7f1efd4e4410>)]

In [51]:
## How to find out the length . 

grouped_by_result = grouped_rdd.map(lambda row :(row[0],len(row[1])))

In [52]:
grouped_by_result.collect()

[('Pune', 23),
 ('Hyderabad', 12),
 ('Mumbai', 9),
 ('Delhi', 9),
 ('Bangalore', 7),
 ('Ahmedabad', 12),
 ('Chennai', 12),
 ('Kolkata', 15)]

In [54]:
!hadoop fs -ls '/user/donnirmal86/'

Found 7 items
drwxr-xr-x   - donnirmal86 hadoop          0 2025-08-08 11:06 /user/donnirmal86/.sparkStaging
-rw-r--r--   2 donnirmal86 hadoop   10528211 2025-08-14 13:27 /user/donnirmal86/customers_10mb.csv
-rw-r--r--   2 donnirmal86 hadoop       5488 2025-08-13 09:41 /user/donnirmal86/first_100_customers.csv
-rw-r--r--   2 donnirmal86 hadoop     836123 2025-08-13 09:43 /user/donnirmal86/items.csv
-rw-r--r--   2 donnirmal86 hadoop     863301 2025-08-13 09:43 /user/donnirmal86/orders.csv
-rw-r--r--   2 donnirmal86 hadoop     854480 2025-08-13 09:44 /user/donnirmal86/payments.csv
-rw-r--r--   2 donnirmal86 hadoop     769971 2025-08-13 09:44 /user/donnirmal86/shippings.csv


In [55]:
hdfs_path = '/user/donnirmal86/customers_10mb.csv'
rdd = spark.sparkContext.textFile(hdfs_path)

In [56]:
#Reduce By Key -1 shot  Data is written 508 bytes 

In [58]:
rdd.map(lambda row : row.split(',')).map(lambda row:(row[2],1)).reduceByKey(lambda x,y:x+y).collect()

[('Pune', 21481),
 ('Hyderabad', 21174),
 ('Delhi', 21123),
 ('Mumbai', 21041),
 ('city', 1),
 ('Bangalore', 21272),
 ('Ahmedabad', 21272),
 ('Chennai', 21046),
 ('Kolkata', 21264)]

In [ ]:
# GroupByKey -1 shot    Data is written 2.3KB 

In [66]:
rdd.map(lambda row: row.split(',')) \
   .map(lambda row: (row[2], 1)) \
   .groupByKey() \
   .map(lambda row: (row[0], len(row[1]))).collect()

[('Pune', 21481),
 ('Hyderabad', 21174),
 ('Delhi', 21123),
 ('Mumbai', 21041),
 ('city', 1),
 ('Bangalore', 21272),
 ('Ahmedabad', 21272),
 ('Chennai', 21046),
 ('Kolkata', 21264)]

In [ ]:
print("hello")